## NYC Demographics and SAT Scores - 2012

SATs have become a controversial topic in the US due to a claim that they are unfair to certain demographic groups.
In this project I will explore relationships between demographics and SAT scores in the NYC in 2012. Because demographics and geographics are often intertwined, as in the case of the NYC, I will also be looking at geographic data.

The data is provided by NYC Open Data. There are eight files I will use. I placed them into a folder titled 'data':

* SAT Scores (2012)
* AP Scores (2010)
* Class Size (2010-2011)
* Attendance (2010-2011)
* Graduation Outcomes (2005-2010)
* School Demographics and Accountability (2006-2012)
* School Survey (2011) excluding district 75 - see below
* School Survey (2011) district 75 - see below

*According to NYC Open Data, school survey is taken each year by students, teachers, and parents to assess the community's opinions on academic expectations, communication, engagement, safety, and respect.

*District 75 is for children with disabilities and cognitive delays.

My first step is to combine all the data files into one dataset.

Here are the common fields (columns) that I will use to combine all the data. Please note that DBN is a combination of 3 pieces of information that can be extracted into its own field (column): district, borough, and school number.
* SAT Scores (2012) <span style="color:red">DBN (Unique school identifier)</span>
* AP Scores (2010) <span style="color:red">DBN (Unique school identifier)</span>
* Class Size (2010-2011) Combination of <span style="color:green">CSD</span> and <span style="color:blue">SCHOOL CODE</span> is the same as the <span style="color:red">DBN (Unique school identifier)</span>
* Attendance (2010-2011) <span style="color:green">District</span>
* Graduation Outcomes (2005-2010) <span style="color:red">DBN (Unique school identifier)</span>
* School Demographics and Accountability (2006-2012) <span style="color:red">DBN (Unique school identifier)</span>
* School Survey (2011) excluding district 75 <span style="color:red">DBN (Unique school identifier)</span>
* School Survey (2011) district 75 <span style="color:red">DBN (Unique school identifier)</span>

### Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [83]:
import re

In [3]:
%matplotlib inline

### Read Files

In [59]:
sat_df = pd.read_csv('schools/sat_results.csv')
ap_df = pd.read_csv('schools/ap_2010.csv')
class_size_df = pd.read_csv('schools/class_size.csv')
attendance_df = pd.read_csv('schools/attendance.csv')
graduation_df = pd.read_csv('schools/graduation.csv')
demographics_df = pd.read_csv('schools/demographics.csv')
hs_directory_df = pd.read_csv('schools/hs_directory.csv')
all_survey = pd.read_csv('schools/survey_all.txt', sep='\t', encoding='cp1252')
d75_survey = pd.read_csv('schools/survey_d75.txt', sep = '\t', encoding='cp1252')

### Add Dataframes to Dictionary

In [60]:
data = {'ap_2010': ap_df,
        'class_size': class_size_df,
        'demographics': demographics_df,
        'graduation': graduation_df,
        'hs_directory': hs_directory_df,
        'sat_results': sat_df,
        'attendance': attendance_df}

### Explore SAT Scores

In [61]:
print(data['sat_results'].head())

      DBN                                    SCHOOL NAME  \
0  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448            UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                     EAST SIDE COMMUNITY SCHOOL   
3  01M458                      FORSYTH SATELLITE ACADEMY   
4  01M509                        MARTA VALLE HIGH SCHOOL   

  Num of SAT Test Takers SAT Critical Reading Avg. Score SAT Math Avg. Score  \
0                     29                             355                 404   
1                     91                             383                 423   
2                     70                             377                 402   
3                      7                             414                 401   
4                     44                             390                 433   

  SAT Writing Avg. Score  
0                    363  
1                    366  
2                    370  
3                    359  
4                    38

In [62]:
data['sat_results'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   DBN                              478 non-null    object
 1   SCHOOL NAME                      478 non-null    object
 2   Num of SAT Test Takers           478 non-null    object
 3   SAT Critical Reading Avg. Score  478 non-null    object
 4   SAT Math Avg. Score              478 non-null    object
 5   SAT Writing Avg. Score           478 non-null    object
dtypes: object(6)
memory usage: 22.5+ KB


In [63]:
data['sat_results'].describe()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
count,478,478,478,478,478,478
unique,478,478,175,164,173,163
top,13K674,BEDFORD ACADEMY HIGH SCHOOL,s,s,s,s
freq,1,1,57,57,57,57


#### It appears that SAT scores are not available for some schools, and I will remove them

In [64]:
sat_df = sat_df[sat_df['Num of SAT Test Takers'] != 's']

#### Now I can convert columns to a numeric type

In [65]:
convert_cols = sat_df.columns[2:]

In [66]:
convert_cols #show columns I am converting from object to numeric

Index(['Num of SAT Test Takers', 'SAT Critical Reading Avg. Score',
       'SAT Math Avg. Score', 'SAT Writing Avg. Score'],
      dtype='object')

In [67]:
for col in convert_cols:
    data['sat_results'][col] = pd.to_numeric(data['sat_results'][col], errors='coerce')

#### Verify that columns converted to numeric

In [68]:
sat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421 entries, 0 to 477
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   DBN                              421 non-null    object
 1   SCHOOL NAME                      421 non-null    object
 2   Num of SAT Test Takers           421 non-null    object
 3   SAT Critical Reading Avg. Score  421 non-null    object
 4   SAT Math Avg. Score              421 non-null    object
 5   SAT Writing Avg. Score           421 non-null    object
dtypes: object(6)
memory usage: 23.0+ KB


#### Continue with the exploration

In [69]:
sat_df.describe()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
count,421,421,421,421,421,421
unique,421,421,174,163,172,162
top,13K674,BEDFORD ACADEMY HIGH SCHOOL,54,398,385,368
freq,1,1,10,8,9,9


#### Everything looks clean with no missing values, with one possible exception. I see that the median number of SAT takers is 62, but maximum is 1,277. I will sort this data and check out top 5 schools by the number of SAT takers

In [70]:
sat_df.sort_values('Num of SAT Test Takers', ascending=False).head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
200,10X477,MARBLE HILL HIGH SCHOOL FOR INTERNATIONAL STUDIES,99,414,435,414
425,28Q680,QUEENS GATEWAY TO HEALTH SCIENCES SECONDARY SC...,99,513,523,502
398,25Q670,ROBERT F. KENNEDY COMMUNITY HIGH SCHOOL,99,431,458,441
441,30Q301,ACADEMY FOR CAREERS IN TELEVISION AND FILM,98,410,440,405
212,11X288,COLLEGIATE INSTITUTE FOR MATH AND SCIENCE,97,444,471,433


#### Based on this, I don't believe the data is bad in the Num of Test Takers column, a small percentage of schools just have a lot of test takers. 
#### Next I will loop through the dataframes and show the first 5 rows of data in each

In [71]:
for dataframe in data.keys():
    print(dataframe)
    print(data[dataframe].head())

ap_2010
      DBN                             SchoolName AP Test Takers   \
0  01M448           UNIVERSITY NEIGHBORHOOD H.S.              39   
1  01M450                 EAST SIDE COMMUNITY HS              19   
2  01M515                    LOWER EASTSIDE PREP              24   
3  01M539         NEW EXPLORATIONS SCI,TECH,MATH             255   
4  02M296  High School of Hospitality Management             NaN   

  Total Exams Taken Number of Exams with scores 3 4 or 5  
0                49                                   10  
1                21                                  NaN  
2                26                                   24  
3               377                                  191  
4               NaN                                  NaN  
class_size
   CSD BOROUGH SCHOOL CODE                SCHOOL NAME GRADE  PROGRAM TYPE  \
0    1       M        M015  P.S. 015 Roberto Clemente     0K       GEN ED   
1    1       M        M015  P.S. 015 Roberto Clemente     0K    

#### Combine two survey files into one

In [72]:
survey = pd.concat([all_survey, d75_survey])

In [73]:
survey.head()

,dbn,bn,schoolname,d75,studentssurveyed,highschool,schooltype,rr_s,rr_t,rr_p,...,s_q14_2,s_q14_3,s_q14_4,s_q14_5,s_q14_6,s_q14_7,s_q14_8,s_q14_9,s_q14_10,s_q14_11
0,01M015,M015,P.S. 015 Roberto Clemente,0,No,0.0,Elementary School,NaN,88,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01M019,M019,P.S. 019 Asher Levy,0,No,0.0,Elementary School,NaN,100,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01M020,M020,P.S. 020 Anna Silver,0,No,0.0,Elementary School,NaN,88,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01M034,M034,P.S. 034 Franklin D. Roosevelt,0,Yes,0.0,Elementary / Middle School,89.0,73,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01M063,M063,P.S. 063 William McKinley,0,No,0.0,Elementary School,NaN,100,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Remove unnecessary data from survey

In [74]:
survey.rename(columns={'dbn': 'DBN'}, inplace=True)

In [75]:
keep_cols = ['DBN', 'rr_s', 'rr_t', 'rr_p', 'N_s', 'N_t', 'N_p', 'saf_p_11', 'com_p_11', 'eng_p_11', 'aca_p_11',
       'saf_t_11', 'com_t_11', 'eng_t_11', 'aca_t_11', 'saf_s_11', 'com_s_11',
       'eng_s_11', 'aca_s_11', 'saf_tot_11', 'com_tot_11', 'eng_tot_11',
       'aca_tot_11']

In [76]:
survey = survey[keep_cols]

#### Continue data transformation and cleanup

In [77]:
data['hs_directory'].rename(columns={'dbn': 'DBN'}, inplace=True)
data['class_size']['padded_csd'] = data['class_size']['CSD'].astype(str).str.zfill(2)
data['class_size']['DBN'] = data['class_size']['padded_csd'] + data['class_size']['SCHOOL CODE']

In [78]:
#hs_directory_df.rename(columns={'dbn': 'DBN'}, inplace=True)
#class_size_df['padded_csd'] = class_size_df['CSD'].astype(str).str.zfill(2)
#class_size_df['DBN'] = class_size_df['padded_csd'] + class_size_df['SCHOOL CODE']

#### Add SAT Scores

In [79]:
#sat_df['sat_score'] = sat_df['SAT Critical Reading Avg. Score'] + sat_df['SAT Math Avg. Score'] + sat_df['SAT Writing Avg. Score']

In [81]:
data['sat_results']['sat_score'] = data['sat_results']['SAT Critical Reading Avg. Score'] + \
    data['sat_results']['SAT Math Avg. Score'] + data['sat_results']['SAT Writing Avg. Score']

In [82]:
data['sat_results'].head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29.0,355.0,404.0,363.0,1122.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91.0,383.0,423.0,366.0,1172.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70.0,377.0,402.0,370.0,1149.0
3,01M458,FORSYTH SATELLITE ACADEMY,7.0,414.0,401.0,359.0,1174.0
4,01M509,MARTA VALLE HIGH SCHOOL,44.0,390.0,433.0,384.0,1207.0


#### Extract coordinates from data

In [112]:
def extract_latitude(string):
    result = re.findall('\(.+\)', string)[0]
    result = result.split(',')[0]
    result = result.replace('(', '')
    return float(result)

In [114]:
#hs_directory_df['lat'] = hs_directory_df['Location 1'].apply(extract_latitude)

In [ ]:
data['hs_directory']['lat'] = data['hs_directory']['Location 1'].apply(extract_latitude)

In [86]:
data['hs_directory'].head()

'8 21 Bay 25 Street\nFar Rockaway, NY 11691\n(40.601989336, -73.762834323)'

In [115]:
def extract_longitude(string):
    result = re.findall('\(.+\)', string)[0]
    result = result.split(',')[1]
    result = result.replace(')', '')
    return float(result)

In [116]:
#hs_directory_df['lon'] = hs_directory_df['Location 1'].apply(extract_longitude)

In [117]:
data['hs_directory']['lon'] = data['hs_directory']['Location 1'].apply(extract_longitude)

In [118]:
data['hs_directory'].head()

,DBN,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,priority10,Location 1,Community Board,Council District,Census Tract,BIN,BBL,NTA,lat,lon
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,NaN,NaN,...,NaN,"8 21 Bay 25 Street\nFar Rockaway, NY 11691\n(4...",14.0,31.0,"100,802","4,300,730","4,157,360,001",Far Rockaway-Bayswater ...,40.601989,-73.762834
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,NaN,NaN,...,NaN,"2630 Benson Avenue\nBrooklyn, NY 11214\n(40.59...",13.0,47.0,306,"3,186,454","3,068,830,001",Gravesend ...,40.593594,-73.984729
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,NaN,NaN,...,NaN,"1014 Lafayette Avenue\nBrooklyn, NY 11221\n(40...",3.0,36.0,291,"3,393,805","3,016,160,001",Stuyvesant Heights ...,40.692134,-73.931503
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,NaN,NaN,...,NaN,"1980 Lafayette Avenue\nBronx, NY 10473\n(40.82...",9.0,18.0,16,"2,022,205","2,036,040,039",Soundview-Castle Hill-Clason Point-Harding Par...,40.822304,-73.855961
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,NaN,NaN,...,NaN,"100 Amsterdam Avenue\nNew York, NY 10023\n(40....",7.0,6.0,151,"1,030,341","1,011,560,030",Lincoln Square ...,40.773671,-73.985269
